In [13]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import getpass

In [14]:
# Load environment variables
load_dotenv()

# Get values from .env
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
key = os.getenv("AZURE_OPENAI_KEY")
deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")


In [15]:
# Create Azure OpenAI client
client = AzureOpenAI(
    api_key=key,
    api_version=api_version,
    azure_endpoint=endpoint
)

# Example input text
input_text = "Hello World"

# Make an embedding request
response = client.embeddings.create(
    model=deployment,
    input=input_text
)

# Get the vector (list of floats)
embedding_vector = response.data[0].embedding

print(f"Embedding length: {len(embedding_vector)}")
print(f"First 5 values: {embedding_vector[:5]}")

Embedding length: 1536
First 5 values: [-0.007019899319857359, 0.0034685435239225626, -0.00714093167334795, -0.029149780049920082, -0.013071536086499691]


In [16]:
import numpy as np

input_1 = "Artificial Intelligence"
input_2 = "AI"
input_3 = "Roman Empire"

reference = "Artificial intelligence"

inputs = [input_1, input_2, input_3]
embeddings = []

for input in inputs:
    response = client.embeddings.create(
        model=deployment,
        input=input
    )
    embedding_vector = response.data[0].embedding
    embeddings.append(embedding_vector)
    
# Get embedding for reference text
ref_response = client.embeddings.create(
    model=deployment,
    input=reference
)
ref_embedding = np.array(ref_response.data[0].embedding)

# Compute cosine similarity
similarities = []
for emb in embeddings:
    emb_vec = np.array(emb)
    cos_sim = np.dot(ref_embedding, emb_vec) / (np.linalg.norm(ref_embedding) * np.linalg.norm(emb_vec))
    similarities.append(cos_sim)

print("Cosine similarities to reference:")
for i, sim in enumerate(similarities):
    print(f"{inputs[i]}: {sim:.4f}")


Cosine similarities to reference:
Artificial Intelligence: 0.9952
AI: 0.8804
Roman Empire: 0.7659


In [ ]:
from langchain.chat_models import init_chat_model

# Initialize chat model
llm = init_chat_model(
    "gpt-35-turbo", 
    model_provider="azure_openai",
    api_key=key,
    api_version=api_version,
)

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Initialize embeddings model
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=key,
    api_version=api_version,
)